# 房屋价格预测项目

## 练习项目

在这个项目中，我还要注重练习新工具的使用：

- Pandas Profiling：自动生成数据集的概览
- Pandas Pipeline: 形成链式方法

In [1]:
import pandas as pd
import pdpipe as pdp

**Pandas Profiling 的使用方法：**
```
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

df = pd.DataFrame(
    np.random.rand(100, 5),
    columns=['a', 'b', 'c', 'd', 'e']
)

profile = ProfileReport(df, title='Pandas Profiling Report')

Saving the report
If you want to generate a HTML report file, save the ProfileReport to an object and use the to_file() function:

profile.to_file('your_report.html')
Alternatively, you can obtain the data as JSON:

# As a string
json_data = profile.to_json()

# As a file
profile.to_file('your_report.json')
```

**对大数据集可以使用这样的方法：**
```
profile = ProfileReport(large_dataset, minimal=True)
profile.to_file('output.html')
```

**还可以直接在命令行中使用：**
pandas_profiling -h

**Pandas Pipeline 的使用方法:**
[这是一篇值得参考的博客](https://www.cnblogs.com/feffery/p/12179647.html)

```
# 以pdp.PdPipeline传入流程列表的方式创建pipeline
'''
pdp.ColDrop:删除列；
ApplyByCols：基于列做，应用apply
RowDrop：删除行
案例解释：
1、删除original_title列
2、对title列进行小写化处理
3、丢掉vote_average小于等于7，且original_language不为en的行
4、求得genres对应电影类型的数量保存为新列genres_num，并删除原有的genres列
5、丢掉genres_num小于等于5的行

这些操作操作直接使用pandas并不会花多少时间，但是如果想要不创造任何中间临时结果一步到位产生所需的数据框子集，
并且保持代码的可读性不是一件太容易的事，但是利用pdpipe，就可以非常优雅地实现上述过程
'''
first_pipeline = pdp.PdPipeline([pdp.ColDrop('original_title'), 
                                 pdp.ApplyByCols(columns=['title'], func=lambda x: x.lower()),
                                 pdp.RowDrop({'vote_average': lambda x: x <= 7, 'original_language': lambda x: x != 'en'}),
                                 pdp.ApplyByCols(columns=['genres'], func=lambda x: [item['name'] for item in eval(x)].__len__(), result_columns=['genres_num']),
                                 pdp.RowDrop({'genres_num': lambda x: x <= 5})])

# 将创建的pipeline直接作用于data直接得到所需结果，并打印流程信息
first_pipeline(data, verbose=True).reset_index(drop=True)
```

## 数据清洗

我们已经在命令行中使用了命令，生成对训练数据机的描述统计，如果没有生成过，可以使用如下命令：

接下来，我们对数据进行清洗，包含以下主要步骤：

<img src='../figs/data_clean.jpg' alt='数据清洗步骤' style='zoom:50%'>

### 选择子集：

此时，参考上一步生成的数据描述文档，和数据集的说明文档，对变量逐一进行检查，旨在理解哪些特征是有信息量的。

根据个人经验，房屋🏠价格影响程度由以下因素决定：

- 区位 (location)：周围环境、交通便利、坡度朝向、
- 户型 (size)：面积大小、庭院大小、规整程度、
- 设施 (facility)：水电、油气、加热、
- 产权 (property)：出售、长租、分配
- 材料 (material)：基本材料、装饰材料
- 评估 (evaluate)：新旧程度、状况评估

In [4]:
train = pd.read_csv('../data/train.csv')
train.head(3)

print("原始的变量:")
train.columns

print("训练数据集大小:")
train.shape

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


原始的变量：


Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

(1460, 81)

In [3]:
# 根据上述大致划分，对每个变量有一个归类

locations = {
    'MSSubClass': 'house_class_cat',  # 售卖类型，分类
    'MSZoning': 'house_zoning_cat',  # 区位，分类
    'LotFrontage': 'adj_street_frontage_num',  # 与街角相连的长度，数字
    'Street': 'street_cat',  # 街道是石子路还是铺砌的，分类
    'Alley': 'alley_cat',  # 门口小路是石子还是铺砌，分类
    'LandContour': 'land_flatness_cat',  # 土地的平坦类型
    'LandSlope': 'land_slope_cat',  # 土地的坡度
    'Neighbourhood': 'neighbourhood_cat',  # 街道类型
    'Condition1': 'location_condition_1_cat',  # 区位状况1
    'Condition2': 'location_condition_2_cat',  # 区位状况2
}

config = {
    'LotArea': 'area_num',  # 房屋面积
    'LotShape': 'shape_cat',  # 房屋形状
    'LotConfig': 'config_cat',  # 房屋结构
    'BldgType': 'dwelling_type_cat',  # 户型
    'HouseStyle': 'house_style_cat',  # 房屋风格
    'RoofStyle': 'roof_style_cat',  # 屋顶的风格
    'MasVnrArea': 'masonry_veneer_area_num',  # 隔板面积
    'BsmtFinSF1': 'basement_finish_1_num',  # 完工的基地面积
    'BsmtFinSF2': 'basement_finish_2_num',  # 完工的基地面积
    'BsmtUnfSF': 'unfinished_basement_num',  # 未完工的面积
    'TotalBsmtSF': 'total_basement_num',  # 总面积
    '1stFlrSF': '1st_floor_num',  # 一楼面积
    '2ndFlrSF': '2nd_floor_num',  # 二楼面积
    'LowQualFinSF': 'low_quality_finished_num',  # 低质量完工的面积
    'GrLivArea': 'living_area_num',  # 居住面积
    'BsmtFullBath': 'basement_full_bathroom_num',  # 高档浴室
    'BsmtHalfBath': 'basement_half_bathroom_num',  # 浴室
    'FullBath': 'full_bathroom_num',  # 总共的高档浴室
    'HalfBath': 'half_bathroom_num',  # 总共的浴室
    'Bedroom': 'bedroom_num',  # 卧室数量
    'Kitchen': 'kitchen_num',  # 厨房数量
    'TotRmsAbvGrd': 'total_rooms_num',  # 总共的房间数量
    'OpenPorchSF': 'open_porch_num',  # 开放门廊的面积
    'EnclosedPorch': 'enclose_proch_num',  # 关闭门廊的面积
    '3SsnPorch': 'season_porch_num',  # 能用三个季节的门廊（应该是冬季不能用？）
    'ScreenPorch': 'screen_porch_num',  # 屏风门廊
    
}

facilities = {
    'Utilities': 'utility_cat',  # 水、电、油、汽等是否有
    'Heating': 'heating_cat',  # 加热设施
    'CentralAir': 'center_air_bool',  # 有没有中央空调
    'Electrical': 'electrical_cat',  # 供电情况
    'Fireplaces': 'fireplaces_num',  # 总共的壁炉数量
    'GarageType': 'garage_type_cat',  # 车库情况
    'GarageYrBlt': 'garage_built_date',  # 车库修建年份
    'GarageFinish': 'garage_finish_cat',  # 车库装修状况
    'GarageArea': 'garage_area_num',  # 车库面积
    'PavedDrive': 'paved_drive_cat',  # 车库路是否铺砌
    'PoolArea': 'pool_area_num',  # 泳池面积
    'Fence': 'fence_cat',  # 围栏情况
    'MiscFeature': 'miscellaneous_cat',  # 没有提到的一些额外杂项设施
    'MiscVal': 'miscellaneous_price_num',  # 没有提到的额外杂项设施的价格
}

sales = {
    'YearBuilt': 'built_date',  # 修建时间
    'YearRemodAdd': 'remod_date',  # 装修时间
    'Functional': 'functional_condition_cat',  # 大部分房屋都是典型功能
    'MoSold': 'sold_month_str',  # 出售月份
    'YrSold': 'sold_year_str',  # 出售年份
    'SaleType': 'sale_type_cat',  # 交易类型
    'SaleCondition': 'sale_condition_cat',  # 交易条件
}

materials = {
    'RoofMatl': 'roof_material_cat',  # 屋顶材料
    'Exterior1st': 'exterior_1st_cat',  # 外部第一层材料
    'Exterior2st': 'exterior_2st_cat',  # 外部第二层材料
    'MasVnrType': 'masonry_veneer_type_cat',   # 隔板材料
    'WoodDeckSF': 'wood_deck_num',  # 木地板面积
}

evaluates = {
    'OverallQual': 'house_quality_cat', 
    'OverallCond': 'house_condition_cat', 
    'ExterQual': 'exterior_material_quality_cat',  # 外部材料的质量
    'ExterCond': 'exterior_material_condition_cat',  # 外部材料的状况
    'BsmtQual': 'basement_quality_cat',  # 基地质量
    'BsmtCond': 'basement_condition_cat',  # 基地状况
    'BsmtExposure': 'basement_exposure_cat',  # 基地采光
    'BsmtFinType1': 'basement_finish_type_1_cat',  # 基地完工类型评级
    'BsmtFinType2': 'basement_finish_type_2_cat',  # 基地完工评级
    'HeatingQC': 'heating_quality_condition_cat',  # 加热质量评级
    'KitchenQual': 'kitchen_quality_cat',  # 厨房质量
    'FireplaceQu': 'fireplace_quality_cat',  # 壁炉质量
    'GarageQual': 'garage_quality_cat',  # 车库质量
    'GarageCond': 'garage_condition_cat',  # 车库情况
    'PoolQC': 'pool_quality_cat',  # 泳池质量
}

variable_types = [locations, config, facilities, sales, materials, evaluates]

对上述变量进行子集选择，因为样本量远远大于特征数量，所以首先尽量保留所有特征。

唯一需要调整的，是把时间年月合起来，然后删除原来不用的两个变量。

In [21]:
# 新增一个列, datetime 类型，是聚合了售出年份和月份的
agg_year_month = lambda row: str(row['YrSold']) + '-' + str(row['MoSold'])

choose_subset = pdp.PdPipeline([
    pdp.ApplyToRows(agg_year_month, colname='sold_time'),  # 新增出售年月
    pdp.ColDrop(['YrSold', 'MoSold']),  # 删除原先的年月
])

choose_subset(train, verbose=True)

- Generating a column with a function .
- Drop columns YrSold, MoSold


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,SaleType,SaleCondition,SalePrice,sold_time
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,WD,Normal,208500,2008-2
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,WD,Normal,181500,2007-5
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,WD,Normal,223500,2008-9
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,WD,Abnorml,140000,2006-2
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,WD,Normal,250000,2008-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,WD,Normal,175000,2007-8
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,WD,Normal,210000,2010-2
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,2500,WD,Normal,266500,2010-5
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,WD,Normal,142125,2010-4


### 变量重命名

- 调整数据类型
- 生成定序变量
- 生成哑变量
- 使用上一步建立的些字典为变量重命名

In [ ]:
# 将数据

for 
"""
       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness
       NA
"""


- MSSubClass: 参与销售的住宅类型应该是Cat数据，而非数字
- LotFrontage: 有17.7%缺失值